In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.multiclass import OneVsOneClassifier
from sklearn import preprocessing

sns.set()
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier

In [ ]:
# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"
!ls "/content/drive/My Drive/Colab Notebooks"

Mounted at /content/drive
 Classroom  'Colab Notebooks'   Videos	 Папка	 проекти
'Copy of featuretools.ipynb'
'Copy of Trees.ipynb'
'Copy of Untitled1 (1).ipynb'
'Copy of Untitled1.ipynb'
 crx.data
 data.csv
 Homework_1.ipynb
 Homework4.ipynb
 Homework5.ipynb
 HW11.ipynb
 HW9.ipynb
'hw_pandas (1).ipynb'
 hw_pandas.ipynb
'player_data (1).csv'
 player_data.csv
'Players (1).csv'
 Players.csv
'Seasons_Stats (1).csv'
 Seasons_Stats.csv
 T10Y2Y.csv
'topic2_practice_visual_titanic_solution (1).ipynb'
 topic2_practice_visual_titanic_solution.ipynb
 TreeHomework.ipynb
 Untitled0.ipynb
 Untitled2.ipynb
 winequality-red.csv
 winequality-white.csv


In [ ]:
X = []
y = []
count_class1 = 0
count_class2 = 0
max_datapoints = 2500

data_class = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data.csv', sep=',', header=None)

data_class.head()

,0,1,2,3,4,5,6,7,8
0,process.b1.capacity,process.b2.capacity,process.b3.capacity,process.b4.capacity,property.price,property.product,property.winner,verification.time,verification.result
1,0,0,2,1,59,1,0,163.31666666666666,False
2,0,0,2,1,59,2,0,200.86,False
3,0,0,2,1,59,4,0,154.88888888888889,False
4,0,0,2,1,59,6,0,108.64,False


In [ ]:
data_class.info()
data_class.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2044 entries, 0 to 2043
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2044 non-null   object
 1   1       2044 non-null   object
 2   2       2044 non-null   object
 3   3       2044 non-null   object
 4   4       2044 non-null   object
 5   5       2044 non-null   object
 6   6       2044 non-null   object
 7   7       2044 non-null   object
 8   8       2044 non-null   object
dtypes: object(9)
memory usage: 143.8+ KB


(2044, 9)

In [ ]:
from pandas.errors import DataError
input_file = '/content/drive/My Drive/Colab Notebooks/data.csv'

with open(input_file, 'r') as f:
  for line in f.readlines():
    if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
      break
    if '?' in line:
      continue
    data = line[:-1].split(',')

    # Останнім елементом кожного
    # рядка є позначка. Залежно від цієї мітки ми відноситимемо дані до того чи
    # іншого класу.

    if data[-1] == 'False' and count_class1 < max_datapoints:
        X.append(data)
        count_class1 += 1
    if data[-1] == 'True' and count_class2 < max_datapoints:
        X.append(data)
        count_class2 += 1

print(X)

[['0', '0', '2', '1', '59', '1', '0', '163.31666666666666', 'False'], ['0', '0', '2', '1', '59', '2', '0', '200.86', 'False'], ['0', '0', '2', '1', '59', '4', '0', '154.88888888888889', 'False'], ['0', '0', '2', '1', '59', '6', '0', '108.64', 'False'], ['0', '0', '2', '1', '60', '1', '0', '85.46666666666667', 'True'], ['0', '0', '2', '1', '60', '1', '3', '82.0', 'True'], ['0', '0', '2', '1', '60', '1', '4', '154.25', 'False'], ['0', '0', '2', '1', '60', '2', '0', '80.95', 'True'], ['0', '0', '2', '1', '60', '2', '3', '78.0', 'True'], ['0', '0', '2', '1', '60', '2', '4', '197.01', 'False'], ['0', '0', '2', '1', '60', '4', '0', '80.25', 'True'], ['0', '0', '2', '1', '60', '4', '3', '77.91666666666667', 'True'], ['0', '0', '2', '1', '60', '4', '4', '144.69444444444446', 'False'], ['0', '0', '2', '1', '60', '6', '0', '88.12', 'True'], ['0', '0', '2', '1', '60', '6', '3', '78.84', 'True'], ['0', '0', '2', '1', '60', '6', '4', '78.15', 'True'], ['0', '0', '2', '1', '69', '3', '0', '112.05', 

In [ ]:
# Перетворення на масив numpy

X = np.array(X)
X

AttributeError: ignored

In [ ]:
# Перетворення рядкових даних на числові
label_encoder = []
X_encoded = np.empty(X.shape)
for i,item in enumerate(X[0]):
  if item.isdigit():
    X_encoded[:, i] = X[:, i]
  else:
    label_encoder.append(preprocessing.LabelEncoder())
    X_encoded[:, i] = label_encoder[-1].fit_transform(X[:,i])
X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)
X

array([[   0,    0,    2, ...,    1,    0,  585],
       [   0,    0,    2, ...,    2,    0,  722],
       [   0,    0,    2, ...,    4,    0,  529],
       ...,
       [   2,    3,    2, ...,    5,    2, 1998],
       [   2,    3,    2, ...,    5,    3, 1872],
       [   2,    3,    2, ...,    5,    4, 1826]])

In [ ]:
# devision dataset on test patr and train [part]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
y_test

AttributeError: ignored

##Bagging


In [ ]:
clf_rf = RandomForestClassifier(max_depth=2, random_state=0).fit(X_test, y_test)

In [ ]:
y_pred_rf = clf_rf.predict(X_test)
list(zip(X_test, y_pred_rf, y_test))

[(array([   2,    1,    2,    0,   70,    3,    0, 1409]), 0, 0),
 (array([   0,    2,    2,    1,   62,    4,    0, 1557]), 0, 0),
 (array([   2,    1,    2,    1,   69,    5,    0, 1856]), 0, 0),
 (array([  1,   1,   2,   0,  75,   6,   0, 558]), 0, 0),
 (array([   2,    1,    2,    1,   75,    6,    0, 1271]), 0, 0),
 (array([ 1,  3,  2,  0, 72,  1,  0, 94]), 0, 0),
 (array([   2,    3,    2,    1,   90,    5,    4, 1826]), 0, 1),
 (array([  1,   2,   2,   1,  65,   4,   0, 125]), 0, 0),
 (array([  1,   1,   2,   0,  62,   6,   0, 221]), 0, 0),
 (array([  0,   3,   1,   1,  70,   3,   4, 685]), 0, 0),
 (array([  1,   2,   2,   1,  71,   3,   0, 473]), 0, 0),
 (array([   1,    1,    2,    1,   80,    2,    1, 1656]), 0, 1),
 (array([   0,    2,    1,    1,   61,    1,    0, 1616]), 0, 0),
 (array([ 1,  2,  2,  0, 69,  1,  0, 40]), 0, 0),
 (array([ 1,  2,  2,  0, 69,  6,  0, 93]), 0, 0),
 (array([  1,   2,   2,   0,  74,   2,   0, 874]), 0, 0),
 (array([   0,    1,    2,    1,   63,  

In [ ]:
print(classification_report(y_test, y_pred_rf, target_names=[f'class_{i}' for i in range(2)]))
print(accuracy_score(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf, normalize=None))

              precision    recall  f1-score   support

     class_0       0.87      1.00      0.93       355
     class_1       1.00      0.02      0.04        54

    accuracy                           0.87       409
   macro avg       0.94      0.51      0.48       409
weighted avg       0.89      0.87      0.81       409

0.8704156479217604
[[355   0]
 [ 53   1]]


 Данная модель научилась хорошо  скор = 0.87
 По матрице ошибок  видим , что  ТР = 355 совпадений, ошибок первого рода нет, ошибок ворого рода 53

In [ ]:
#[хочу посмотреть какой будет результат моей модели на  данных которые она не видела]
y_pred_rf_validate = clf_rf.predict(X_train)
list(zip(X_train, y_pred_rf_validate, y_train))


[(array([   1,    2,    2,    0,   80,    3,    3, 1417]), 0, 0),
 (array([  0,   1,   2,   0,  59,   6,   0, 462]), 0, 0),
 (array([  2,   2,   2,   0,  80,   2,   3, 878]), 0, 0),
 (array([ 1,  3,  2,  1, 64,  4,  0, 82]), 0, 0),
 (array([  2,   1,   2,   1,  76,   6,   0, 955]), 0, 0),
 (array([   0,    2,    2,    0,   63,    1,    0, 1414]), 0, 0),
 (array([   1,    3,    2,    1,   73,    5,    0, 1843]), 0, 0),
 (array([   0,    2,    2,    0,   61,    4,    0, 1355]), 0, 0),
 (array([  0,   3,   2,   1,  63,   2,   0, 461]), 0, 0),
 (array([  2,   1,   2,   0,  70,   2,   0, 744]), 0, 0),
 (array([   2,    3,    2,    0,   76,    3,    0, 1346]), 0, 0),
 (array([  1,   2,   2,   1,  78,   2,   0, 913]), 0, 0),
 (array([  2,   2,   2,   0,  90,   1,   0, 771]), 0, 1),
 (array([   2,    1,    2,    1,   73,    5,    0, 1852]), 0, 0),
 (array([   0,    1,    2,    0,   70,    3,    2, 2006]), 0, 1),
 (array([  1,   3,   2,   0,  61,   2,   0, 901]), 0, 0),
 (array([   0,    3,    

In [ ]:
print(classification_report(y_train, y_pred_rf_validate, target_names=[f'class_{i}' for i in range(2)]))
print(accuracy_score(y_train, y_pred_rf_validate))
print(confusion_matrix(y_train, y_pred_rf_validate, normalize=None))

              precision    recall  f1-score   support

     class_0       0.87      1.00      0.93      1426
     class_1       1.00      0.00      0.01       208

    accuracy                           0.87      1634
   macro avg       0.94      0.50      0.47      1634
weighted avg       0.89      0.87      0.81      1634

0.8733170134638923
[[1426    0]
 [ 207    1]]


Модель работает хорошо и на данных которые не видила

##Stacking

In [ ]:
estimators = [
  ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42)))
  ]
clf_stck = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [ ]:
clf_stck = clf_stck.fit(X_train, y_train)
y_pred_stck = clf_stck.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_stck, target_names=[f'class_{i}' for i in range(2)]))
print(accuracy_score(y_test, y_pred_stck))
print(confusion_matrix(y_test, y_pred_stck, normalize=None))

              precision    recall  f1-score   support

     class_0       0.98      1.00      0.99       355
     class_1       0.98      0.89      0.93        54

    accuracy                           0.98       409
   macro avg       0.98      0.94      0.96       409
weighted avg       0.98      0.98      0.98       409

0.9828850855745721
[[354   1]
 [  6  48]]


##Boosting

In [ ]:
clf_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

In [ ]:
y_pred_gb = clf_gb.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_gb, target_names=[f'class_{i}' for i in range(2)]))
print(accuracy_score(y_test, y_pred_gb))

              precision    recall  f1-score   support

     class_0       0.97      0.98      0.98       355
     class_1       0.85      0.83      0.84        54

    accuracy                           0.96       409
   macro avg       0.91      0.91      0.91       409
weighted avg       0.96      0.96      0.96       409

0.9584352078239609


In [ ]:
clf_lgbm = LGBMClassifier().fit(X_train, y_train)
y_pred_lgbm = clf_lgbm.predict(X_test)
print(classification_report(y_test, y_pred_lgbm, target_names=[f'class_{i}' for i in range(2)]))
print(accuracy_score(y_test, y_pred_lgbm))
print(confusion_matrix(y_test, y_pred_lgbm, normalize=None))

              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00       355
     class_1       1.00      1.00      1.00        54

    accuracy                           1.00       409
   macro avg       1.00      1.00      1.00       409
weighted avg       1.00      1.00      1.00       409

1.0
[[355   0]
 [  0  54]]


In [ ]:

clf_lgbm = LGBMClassifier().fit(X_test, y_test)
y_pred_lgbm = clf_lgbm.predict(X_train)
print(classification_report(y_train, y_pred_lgbm, target_names=[f'class_{i}' for i in range(2)]))
print(accuracy_score(y_train, y_pred_lgbm))
print(confusion_matrix(y_train, y_pred_lgbm, normalize=None))

              precision    recall  f1-score   support

     class_0       0.97      0.99      0.98      1426
     class_1       0.93      0.80      0.86       208

    accuracy                           0.97      1634
   macro avg       0.95      0.89      0.92      1634
weighted avg       0.97      0.97      0.97      1634

0.966952264381885
[[1414   12]
 [  42  166]]


In [ ]:
Все модели дали хороший резудьтат.
Однако я вижу так жк что модели в которых я использовала StackingClassifier в Staikng и GradientBoostingClassifier и LGBMClassifier дали лучший результат, чем RandomForestClassifier в Bagging
Я попробовала натренировать модел с помощью LGBMClassifier на тестовых и трейнинговых данных

Если посмотреть на
X_train.shape
и
X_test.shape то

In [ ]:
X_train.shape

(1634, 8)

In [ ]:
X_test.shape

(409, 8)

X_train больше нежели X_test

Поэтому если тренировать выборку на X_train то результат предикта будет  лучше.
если я учу модель на выборке X_test, y_test она дает очень хорошие результаты тоже
 НО у меня Вопрос!!!!. LGBMClassifier показала результат скора =1
то есть она предиктнула по X_train все данные без ошибок первого и второго рода.
 (Или я что то не так понимаю).
 Это разве не являеться вариантом когда наша модель переучена и даст на новых данных не хороший результ? Или я просто допустила какую-то ошибку?